In [24]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
# load the data first
data= np.loadtxt("./data/data_w3_ex2.csv",delimiter=",")


In [8]:
x=data[:,:-1]
y=data[:,-1]

y=np.expand_dims(y,axis=1)


In [10]:
print(x.shape)
print(y.shape)

(200, 2)
(200, 1)


In [12]:
# Lets split the data into the 3 datasets

x_train,x_,y_train,y_ = train_test_split(x,y,test_size=0.4,random_state=1)
x_cv,x_test,y_cv,y_test= train_test_split(x_,y_,test_size=0.50 , random_state=1)
del x_,y_

In [25]:
# scale the features

scale= StandardScaler()
x_train_scaled = scale.fit_transform(x_train)
x_cv_scaled  = scale.transform(x_cv)
x_test_scaled = scale.transform(x_test)

In [26]:
print(x_train[:2],x_train_scaled[:2])

[[7964.8241206   603.01507538]
 [5070.35175879 2487.43718593]] [[ 0.93567396 -1.21948848]
 [-0.19047806  0.10014556]]


In [20]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_models():
    
    tf.random.set_seed(20)
    
    model_1 = Sequential(
        [
            Dense(25, activation = 'relu'),
            Dense(15, activation = 'relu'),
            Dense(1, activation = 'linear')
        ],
        name='model_1'
    )

    model_2 = Sequential(
        [
            Dense(20, activation = 'relu'),
            Dense(12, activation = 'relu'),
            Dense(12, activation = 'relu'),
            Dense(20, activation = 'relu'),
            Dense(1, activation = 'linear')
        ],
        name='model_2'
    )

    model_3 = Sequential(
        [
            Dense(32, activation = 'relu'),
            Dense(16, activation = 'relu'),
            Dense(8, activation = 'relu'),
            Dense(4, activation = 'relu'),
            Dense(12, activation = 'relu'),
            Dense(1, activation = 'linear')
        ],
        name='model_3'
    )
    
    model_list = [model_1, model_2, model_3]
    
    return model_list


In [37]:
# Import models
# Compile them
# Fit Them
# Make Predictions

# maybe Don't Use this one.

training_err = []
cvs_err = []

models= build_models()

for model in models : 
    model.compile(
        loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
    )
    model.fit(
        x_train_scaled,y_train,epochs=200,verbose=0
    )

    yHat = model.predict(x_train_scaled)
    yHat = tf.math.sigmoid(yHat)
    predictions = np.where(yHat >=0.5 , 1,0)
    err = np.mean(predictions!=y_train)
    training_err.append(err)

    yHat = model.predict(x_cv_scaled)
    yHat = tf.math.sigmoid(yHat)
    predictions = np.where(yHat >=0.5,1,0)
    err = np.mean(predictions!=y_cv)
    cvs_err.append(err)
    


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  


In [40]:
# checking something

# copy pasted this and found out my Learning Rate was high
nn_train_error = []
nn_cv_error = []

models= build_models()
# Loop over each model
for model in models:
    
    # Setup the loss and optimizer
    model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    )

    print(f"Training {model.name}...")

    # Train the model
    model.fit(
           x_train_scaled, y_train,
        epochs=200,
        verbose=0
    )
    
    print("Done!\n")
    
    # Set the threshold for classification
    threshold = 0.5
    
    # Record the fraction of misclassified examples for the training set
    yhat = model.predict(x_train_scaled)
    yhat = tf.math.sigmoid(yhat)
    yhat = np.where(yhat >= threshold, 1, 0)
    train_error = np.mean(yhat != y_train)
    nn_train_error.append(train_error)

    # Record the fraction of misclassified examples for the cross validation set
    yhat = model.predict(x_cv_scaled)
    yhat = tf.math.sigmoid(yhat)
    yhat = np.where(yhat >= threshold, 1, 0)
    cv_error = np.mean(yhat != y_cv)
    nn_cv_error.append(cv_error)

# Print the result
for model_num in range(len(nn_train_error)):
    print(
        f"Model {model_num+1}: Training Set Classification Error: {nn_train_error[model_num]:.5f}, " +
        f"CV Set Classification Error: {nn_cv_error[model_num]:.5f}"
        )

Training model_1...
Done!

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Training model_2...
Done!

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
Training model_3...
Done!

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
Model 1: Training Set Classification Error: 0.02500, CV Set Classification Error: 0.17500
Model 2: Training Set Classification Error: 0.05833, CV Set Classification Error: 0.12500
Model 3: Training Set Classification Error: 0.05000, CV Set Classification Error: 0.15000


In [38]:
# Error in both 
for i in range (len(models)) : 
    print(f"Err in Training : {training_err[i]}")
    print(f"Err in DEV Set : {cvs_err[i]}\n")

Err in Training : 0.058333333333333334
Err in DEV Set : 0.125

Err in Training : 0.03333333333333333
Err in DEV Set : 0.15

Err in Training : 0.025
Err in DEV Set : 0.125



In [39]:
# Chose the model with the less Err 
modelll = np.argmin(cvs_err) + 1
print(modelll)

1


In [43]:
model_num = 3

# Compute the test error
yhat = models[model_num-1].predict(x_test_scaled)
yhat = tf.math.sigmoid(yhat)
yhat = np.where(yhat >= threshold, 1, 0)
nn_test_error = np.mean(yhat != y_test)

print(f"Selected Model: {model_num}")
print(f"Training Set Classification Error: {nn_train_error[model_num-1]:.4f}")
print(f"CV Set Classification Error: {nn_cv_error[model_num-1]:.4f}")
print(f"Test Set Classification Error: {nn_test_error:.4f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
Selected Model: 3
Training Set Classification Error: 0.0500
CV Set Classification Error: 0.1500
Test Set Classification Error: 0.1750
